### SXDG Case

In [1]:
import qiskit

from qiskit_aer import AerSimulator

from rivet_transpiler import transpile
from rivet_transpiler import transpile_left
from rivet_transpiler import get_sinusoids
from rivet_transpiler import get_litmus_circuit

from qiskit_ibm_runtime.fake_provider import fake_backend

from qiskit_ibm_runtime.fake_provider import FakeLimaV2
from qiskit_ibm_runtime.fake_provider import FakeGuadalupeV2
from qiskit_ibm_runtime.fake_provider import FakeMontrealV2

In [2]:
cd ..

/mnt/c/Users/mohor/Jupyter/haiqu


In [3]:
from tests.integration import get_circuits_to_compare
from tests.integration import run_circuits_to_compare
from tests.integration import plot_results_to_compare
from tests.integration import check_delta

In [4]:
# Parameters

OPTIMIZATION_LEVEL = 3

SHOTS_COUNT = 1000
SEED_TRANSPILER = 1234

LAYOUT_METHOD = None
ROUTING_METHOD = None

In [5]:
QUBITS_COUNT = 5

litmus_circuit = get_litmus_circuit(QUBITS_COUNT, "Litmus")

litmus_circuit.draw()

┌──────────────┐                    ┌───┐ Litmus 
Litmus_0_0: ┤ Rz(Litmus_0) ├──■─────────────────┤ X ├───░────
            ├──────────────┤┌─┴─┐               └─┬─┘   ░    
Litmus_0_1: ┤ Rz(Litmus_1) ├┤ X ├──■──────────────┼─────░────
            ├──────────────┤└───┘┌─┴─┐            │     ░    
Litmus_0_2: ┤ Rz(Litmus_2) ├─────┤ X ├──■─────────┼─────░────
            ├──────────────┤     └───┘┌─┴─┐       │     ░    
Litmus_0_3: ┤ Rz(Litmus_3) ├──────────┤ X ├──■────┼─────░────
            ├──────────────┤          └───┘┌─┴─┐  │     ░    
Litmus_0_4: ┤ Rz(Litmus_4) ├───────────────┤ X ├──■─────░────
            └──────────────┘               └───┘        ░

In [6]:
fake_backend = FakeMontrealV2()
# fake_backend = FakeGuadalupeV2()
# fake_backend = FakeLimaV2()


backend = AerSimulator.from_backend(fake_backend)

backend

AerSimulator('aer_simulator_from(fake_montreal)'
             noise_model=<NoiseModel on ['id', 'cx', 'sx', 'measure', 'reset', 'x']>)

In [7]:
backend.options.noise_model = None

In [8]:
QUBITS_COUNT = litmus_circuit.num_qubits

qubits = list(range(QUBITS_COUNT))

# Measurement Part

measurement_circuit = qiskit.QuantumCircuit(QUBITS_COUNT, QUBITS_COUNT)
measurement_circuit.measure(qubits, qubits)

# Transpile Left

transpiled_measurement_circuit = transpile(
    measurement_circuit,
    backend,
    layout_method=LAYOUT_METHOD,
    routing_method=ROUTING_METHOD,
    seed_transpiler=SEED_TRANSPILER,
    optimization_level=OPTIMIZATION_LEVEL
)

transpiled_left_circuit = transpile_left(
    central_circuit=transpiled_measurement_circuit,
    left_circuit=litmus_circuit,
    backend=backend,
    layout_method=LAYOUT_METHOD,
    routing_method=ROUTING_METHOD,
    seed_transpiler=SEED_TRANSPILER,
    optimization_level=OPTIMIZATION_LEVEL
)

transpiled_left_circuit.draw(fold=-1, idle_wires=False)

┌───┐                                                                                                                                                ░          ┌─┐   
  ancilla_6 -> 4 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──┤ X ├──■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░──────────┤M├───
                                                                                                                                                                                                                       ┌───┐     ┌─┴─┐└─┬─┘┌─┴─┐                                                                                                                                           ░          └╥┘   
  ancilla_9 -> 7 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──┤ X ├──■──┤ X ├──■──┤ X ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╫────
                                                                                                                                                                                                        ┌───┐     ┌─┴─┐└─┬─┘┌─┴─┐└───┘     └───┘                                                                                                                                                       ║    
ancilla_11 -> 10 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──┤ X ├──■──┤ X ├──■──┤ X ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╫────
                                                                                                                                            ┌───┐                                        ┌───┐     ┌─┴─┐└─┬─┘┌─┴─┐└───┘     └───┘                                                                                                                                                                      ║    
Litmus_0_3 -> 12 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────┤ X ├──■────────────────────────■─────────■──┤ X ├──■──┤ X ├──■──┤ X ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╫────
                                                                                                              ┌───┐            ┌─┴─┐        └─┬─┘┌─┴─┐┌──────────────────┐┌─┴─┐     ┌─┴─┐└─┬─┘┌─┴─┐└───┘     └───┘                                                                                                                                                              ┌───┐      ░ ┌─┐       ║    
Litmus_0_4 -> 13 ──────────────────────────────────────────────────────────────────────────────────────────■──┤ X ├──■─────────┤ X ├──────────■──┤ X ├┤ Rz(1.0*Litmus_3) ├┤ X ├──■──┤ X ├──■──┤ X ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──┤ X ├──■───░─┤M├───────╫────
                                                                                               ┌───┐     ┌─┴─┐└─┬─┘┌─┴─┐┌──────┴───┴───────┐     └───┘└──────────────────┘└───┘┌─┴─┐├───┤     ├───┤                                                                 

In [9]:
# Assign Parameters

for index, parameter in enumerate(transpiled_left_circuit.parameters):

    transpiled_left_circuit.assign_parameters({parameter: index}, inplace=True)

# Backend

if backend is None:
    run_backend = AerSimulator()
    run_circuit = transpile(transpiled_left_circuit, run_backend)
else:
    run_backend = backend
    run_circuit = transpiled_left_circuit

# Run

job = run_backend.run(run_circuit, shots=SHOTS_COUNT)

counts = job.result().get_counts()

counts

AerError: 'unknown instruction: sxdg'